In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd

# vectorization
from sklearn.preprocessing import LabelEncoder

# misc
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline

# models
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC

In [ ]:
# load train and test data

train = pd.read_csv('/kaggle/input/titanic/train.csv')
test = pd.read_csv('/kaggle/input/titanic/test.csv')

## 1. Structure of the Data

Size of **training examples**: (891,12) <br/>
Size of **test data**: (418,11)

In [ ]:
print(f'Train dataset size: {train.shape}')
train.head()

In [ ]:
print(f'Test dataset size: {test.shape}')
test.head()

## 2. Missing Values

In [ ]:
def missing_values(df, columns):
    """ Finds number of rows where specified columns are missing values.
    
    Args:
        df:
            The dataframe to be analyzed.
        columns:
            The list of columns of the dataframe.
    
    Returns:
         A dictionary with the columns and the number of values they are missing.
    """
    missing = {}
    
    for column in columns:
        total = df[column].value_counts().sum()
        missing[column] = df.shape[0] - total
    
    return missing

In [ ]:
missing = missing_values(train, train.columns)
print(f'Total values that each column should have: {train.shape[0]}')
print(f'Total values that each column is missing:')
missing

## 3. Preprocess

### 3.1 PassengerId

The **PassengerId** column has unique values for each row in the trainning examples. Thus, it won't help in classification.

In [ ]:
train = train.drop(['PassengerId'], axis=1)

### 3.2 Name

The **Name** column has unique values for each row in the trainning examples. Thus, it won't help in classification. <br/>
However, we can create an new feature from it, called **Title** which will be the title of each person.

In [ ]:
train['Title'] = train['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
train = train.drop(['Name'], axis=1)

In [ ]:
test['Title'] = test['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
test = test.drop(['Name'], axis=1)

### 3.3 Age

The **Age** column is missing a lot of values. My intuition is that the age of a person played an important role in his survival and so we'll fill them using the mean of the ages with respect to the persons title. Also, we'll create ranges of the age so it is discretised.

In [ ]:
train['Age'] = train['Age'].fillna(train.groupby('Title')['Age'].transform('median'))
train['Age'] = pd.cut(train['Age'], bins=10)

In [ ]:
test['Age'] = test['Age'].fillna(test.groupby('Title')['Age'].transform('median'))
test['Age'] = pd.cut(test['Age'], bins=10)

### 3.4 Ticket

In [ ]:
print(f'Ticket column has {len(train.Ticket.unique())} different values.')

The **Ticket** column has a lot of different values. Specifically, 681/891 different values. So, we'll drop this column.

In [ ]:
train = train.drop(['Ticket'], axis=1)

In [ ]:
test = test.drop(['Ticket'], axis=1)

### 3.5 Fare

In [ ]:
print(f'Fare column has {len(train.Fare.unique())} different values.')

The **Fare** column has a some unique values (248/891). We'll not drop this column, but we'll create ranges, so it becomes categorical.

In [ ]:
train['Fare'] = pd.cut(train['Fare'], bins=5)

In [ ]:
test['Fare'] = pd.cut(test['Fare'], bins=5)

### 3.6 Embarked

The **Embarked** column has 2 missing values. We'll not remove these rows but we'll fill them with the most frequent value.

In [ ]:
# get the most frequent value in 'Embarked' column
val = train['Embarked'].value_counts().idxmax()

# replace null values
train['Embarked'] = train['Embarked'].fillna(value=val)

In [ ]:
val = test['Embarked'].value_counts().idxmax()
test['Embarked'] = test['Embarked'].fillna(value=val)

### 3.7 Cabin

The **Cabin** column has a lot of missing values. We'll only keep the first letter of the cabin and create a new cabin class, namely 'N' for these rows that have missing values in this column.


**Note:** Some people had two cabins! 

In [ ]:
# keep the first letter from the cabin
train['Cabin'] = train['Cabin'].str[:1]

# fill missing data in "Cabin" column using 'N' (we'll assume that n is a cabin class)
train['Cabin'] = train['Cabin'].fillna(value='N')

In [ ]:
test['Cabin'] = test['Cabin'].str[:1]
test['Cabin'] = test['Cabin'].fillna(value='N')

### 3.8 Alone

The **SibSp** column shows the number of siblings/spouses of a person aboard the Titanic. Similarly, **Parch** shows the number of parents/children of a person aboard the Titanic.

My intuition is that if a person wasn't alone on the Titanic he could have help from his family, or alternatively he could have sacrificed himself/herself to help them (lower chances of survival). So, we'll can combine them in a single feature called **Alone**, which takes 0/1 values, and drop **SibSp** and **Parch**.

In [ ]:
train['Alone'] = 1
train.loc[(train['SibSp'] >= 1) | (train['Parch'] >= 1), 'Alone'] = 0

# drop SibSp and Parch
train = train.drop(['SibSp', 'Parch'], axis=1)

In [ ]:
test['Alone'] = 1
test.loc[(test['SibSp'] >= 1) | (test['Parch'] >= 1), 'Alone'] = 0
test = test.drop(['SibSp', 'Parch'], axis=1)

In [ ]:
print('Training examples after preprocessing:')
train.head()

In [ ]:
print('Test data after preprocessing:')
test.head()

## 4. Vectorize

We'll try categorical encoding vectors.

In [ ]:
train_enc_cat = train.apply(LabelEncoder().fit_transform)

In [ ]:
print(f'Categorical encoding vector size: {train_enc_cat.shape}')
train_enc_cat.head()

## 5. Fitting Models and Score

In [ ]:
y_train = train_enc_cat.Survived
X_train = train_enc_cat.drop(['Survived'], axis=1)

In [ ]:
print(f'Train data size: {X_train.shape}')
X_train.head()

In [ ]:
print(f'Train labels size: {y_train.shape}')
y_train.head()

### 5.1 Logistic Regression

In [ ]:
lr = LogisticRegression()

# cross validation
scores = cross_val_score(lr, X_train, y_train, cv=10)

# accuracy
print(f'Accuracy of Logistic Regression: {scores.mean()}')

### 5.2 Decision Tree

In [ ]:
dt = DecisionTreeClassifier()

# cross validation
scores = cross_val_score(dt, X_train, y_train, cv=10)

# accuracy
print(f'Accuracy for Decision Tree: {scores.mean()}')

### 5.3 Random Forest

In [ ]:
# 100 decision tree classifiers
# feture selection with gini
# decision trees have max depth
rf = RandomForestClassifier()

# cross validation
scores = cross_val_score(rf, X_train, y_train, cv=10)

# accuracy
print(f'Accuracy for Random Forest: {scores.mean()}')

### 5.4 Support Vector Machines

In [ ]:
# values will be scaled using equation (val-mean)/std
svc = make_pipeline(StandardScaler(), LinearSVC())

# cross validation
scores = cross_val_score(svc, X_train, y_train, cv=10)

# accuracy
print(f'Accuracy for Linear SVC: {scores.mean()}')

### 6. Predict on Test

### 6.1 Vectorize

In [ ]:
# test data should have the same format (columns) as training examples
columns = X_train.columns

In [ ]:
test_enc_cat = test[columns].apply(LabelEncoder().fit_transform)

X_test = test_enc_cat

### 6.2 Predict with best model

Best model up to now is logistic regression classifier, with training accuracy 77%.

In [ ]:
best_model = lr

# train model
best_model.fit(X_train, y_train)

# predict on test data
y_pred = best_model.predict(X_test)

## 8 Submission

In [ ]:
submission = pd.DataFrame()
submission['PassengerId'] = test['PassengerId']
submission['Survived'] = y_pred

In [ ]:
print(f'Submission size: {submission.shape}')
submission.head()

In [ ]:
submission.to_csv('/kaggle/working/logistic-regression.csv', index=False)